# 第四章 部署

部署是将我们的模型用于实际应用的关键一步。部署的方式有很多，本章我们主要讨论两种常用的形式：定时任务和API。

## 定时任务

定时任务就是按一定规定时间或者间隔一定时间运行我们的程序，例如每天早上9点运行，每小时运行一次等。这里我们将介绍2个中方法，一种是在`Linux`系统下，使用`crontab`方法；另一种方法是使用`apscheduler`库。

### crontab

`crontab`可以用于设置周期性被执行的指令。可以在`Linux`系统中键入`crontab`命令，使用`crontab`。

~~~bash
crontab -e
~~~

### apscheduler